<a href="https://colab.research.google.com/github/EureXaAI/EurexaBook/blob/main/dm_control/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **`dm_control` tutorial**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-deepmind/dm_control/blob/main/tutorial.ipynb)








> <p><small><small>Copyright 2020 The dm_control Authors.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

This notebook provides an overview tutorial of DeepMind's `dm_control` package, hosted at the [google-deepmind/dm_control](https://github.com/google-deepmind/dm_control) repository on GitHub.

It is adjunct to this [tech report](http://arxiv.org/abs/2006.12983).

**A Colab runtime with GPU acceleration is required.** If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".

这本笔记本提供了一个关于 DeepMind 的 dm_control 包的概览教程，该包托管在 GitHub 上的 google-deepmind/dm_control 仓库中。

它是这篇技术报告的补充材料。

**需要使用带有 GPU 加速的 Colab 运行。**如果你当前使用的是仅支持 CPU 的运行时，可以通过菜单 "Runtime > Change runtime type" 进行切换。

### Installing `dm_control` on Colab: 在 Colab 上安装 `dm_control`

In [ ]:
!nvidia-smi

In [ ]:
#@title Run to install MuJoCo and `dm_control`: 安装 Mujoco 和 `dm_control`
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
# 添加一个 ICD 配置，以便 glvnd 能够识别 Nvidia 的 EGL 驱动。
# 这个配置通常是作为 Nvidia 驱动包的一部分安装的，但 Colab 的内核并不是通过 APT 安装驱动，
# 因此缺少了这个 ICD 配置。
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# 安装 dm_control, 版本大于等于 1.0.28
print('Installing dm_control...')
!pip install -q dm_control>=1.0.28

# Configure dm_control to use the EGL rendering backend (requires GPU)
# 配置 dm_control 以使用 EGL 渲染后端（需要 GPU）
# MUJOCO_GL 是 MuJoCo 用来决定图形渲染方式的环境变量。可选值包括：
# 1. osmesa：CPU 离屏渲染，不依赖 GPU（最兼容，但速度慢）
# 2. egl：使用 GPU 的离屏渲染，高性能，适合无窗口的环境（如 Colab）
# 3. glfw：需要有显示器的本地窗口渲染（适合本地 GUI 使用）
%env MUJOCO_GL=egl

print('Checking that the dm_control installation succeeded...')
try:
  from dm_control import suite
  env = suite.load('cartpole', 'swingup')
  pixels = env.physics.render()
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')
else:
  del pixels, suite

!echo Installed dm_control $(pip show dm_control | grep -Po "(?<=Version: ).+")

# Imports: 导入依赖库

Run both of these cells:

In [ ]:
#@title All `dm_control` imports required for this tutorial: 本教程所需的所有 dm_control 导入

# The basic mujoco wrapper.
from dm_control import mujoco

# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib

# PyMJCF
from dm_control import mjcf

# Composer high level imports
from dm_control import composer
from dm_control.composer.observation import observable
from dm_control.composer import variation

# Imports for Composer tutorial example
from dm_control.composer.variation import distributions
from dm_control.composer.variation import noises
from dm_control.locomotion.arenas import floors

# Control Suite
from dm_control import suite

# Run through corridor example
from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.locomotion.arenas import corridors as corridor_arenas
from dm_control.locomotion.tasks import corridors as corridor_tasks

# Soccer
from dm_control.locomotion import soccer

# Manipulation
from dm_control import manipulation

In [ ]:
#@title Other imports and helper functions: 导入其他包和辅助函数

# General: 通用包
import copy
import os
import itertools
from IPython.display import clear_output
import numpy as np

# Graphics-related: 图形相关包
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL.Image
# Internal loading of video libraries.: 内部加载视频库

# Use svg backend for figure rendering: 使用 SVG 后端进行图像渲染
%config InlineBackend.figure_format = 'svg'

# Font sizes: 字体设置
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Inline video helper function: 内联视频辅助函数
if os.environ.get('COLAB_NOTEBOOK_TEST', False):
  # We skip video generation during tests, as it is quite expensive.
  display_video = lambda *args, **kwargs: None
else:
  def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

# Seed numpy's global RNG so that cell outputs are deterministic. We also try to
# use RandomState instances that are local to a single cell wherever possible.
# 为了使单元格输出具有确定性，给 numpy 的全局随机数生成器设定种子。
# 同时我们也尽可能在每个单元格中使用局部的 RandomState 实例。
np.random.seed(907)

# Model definition, compilation and rendering: 模型定义、编译与渲染



We begin by describing some basic concepts of the [MuJoCo](http://mujoco.org/) physics simulation library, but recommend the [official documentation](http://mujoco.org/book/) for details.

Let's define a simple model with two geoms and a light.

我们首先介绍 MuJoCo 物理仿真库的一些基本概念，但建议查阅官方文档以获取详细信息。

现在我们来定义一个包含两个几何体和一个光源的简单模型。

In [ ]:
#@title A static model: 一个静态模型 {vertical-output: true}

static_model = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
# 从 xml 读取物理模型
physics = mujoco.Physics.from_xml_string(static_model)
# 物理模型渲染
pixels = physics.render()
# 显示图像
PIL.Image.fromarray(pixels)

`static_model` is written in MuJoCo's XML-based [MJCF](http://www.mujoco.org/book/modeling.html) modeling language. The `from_xml_string()` method invokes the model compiler, which instantiates the library's internal data structures. These can be accessed via the `physics` object, see below.

`static_model` 是用 MuJoCo 基于 XML 的 MJCF 建模语言编写的。
`from_xml_string()` 方法会调用模型编译器，该编译器会实例化库的内部数据结构。
这些结构可以通过 `physics` 对象访问，详见下文。

## Adding DOFs and simulating, advanced rendering: 添加自由度并进行仿真, 高级渲染
This is a perfectly legitimate model, but if we simulate it, nothing will happen except for time advancing. This is because this model has no degrees of freedom (DOFs). We add DOFs by adding **joints** to bodies, specifying how they can move with respect to their parents. Let us add a hinge joint and re-render, visualizing the joint axis.

这是一个完全合法的模型，但如果我们对其进行仿真，除了时间推进之外不会发生任何事情。
这是因为该模型没有自由度（DOFs）， 我们可以通过向物体添加**关节**来增加自由度，从而指定它们相对于父物体的运动方式。
下面我们添加一个铰链关节并重新渲染，同时可视化关节轴。

In [ ]:
#@title A child body with a joint: 一个带有铰链关节的子物体 { vertical-output: true }

swinging_body = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
      <geom name="blue_box" type="box" pos="0 .2 .2" size=".05 .05 .05" rgba="0 0 1 1"/>
    </body>
  </worldbody>
</mujoco>
"""
# 从 xml 读取物理模型
physics = mujoco.Physics.from_xml_string(swinging_body)
# 配置渲染中显示什么内容
scene_option = mujoco.wrapper.core.MjvOption()
# 启用渲染中对“关节”的可视化显示
# 如果设置为 False 则不会显示蓝色的铰链关节
scene_option.flags[enums.mjtVisFlag.mjVIS_JOINT] = True
# 使用指定的 scene_option 渲染当前模拟场景，并返回图像像素
pixels = physics.render(scene_option=scene_option)
# 显示图像
PIL.Image.fromarray(pixels)

The things that move (and which have inertia) are called *bodies*. The body's child `joint` specifies how that body can move with respect to its parent, in this case `box_and_sphere` w.r.t the `worldbody`.

Note that the body's frame is **rotated** with an `euler` directive, and its children, the geoms and the joint, rotate with it. This is to emphasize the local-to-parent-frame nature of position and orientation directives in MJCF.

Let's make a video, to get a sense of the dynamics and to see the body swinging under gravity.

会运动的物体（具有惯性）被称为 body（刚体）。刚体的子元素 joint（关节）指定了该刚体相对于其父物体的运动方式，在本例中是 box_and_sphere 相对于 worldbody 的运动。

注意，刚体的坐标系通过 euler 指令进行了**旋转**，其子元素——包括几何体（geoms）和关节（joint）——也随之旋转。这强调了 MJCF 中位置和方向指令是相对于父坐标系定义的这一特性。

接下来我们制作一个视频，以便了解该系统的动力学行为，并观察刚体在重力作用下的摆动。

In [ ]:
#@title Making a video: 视频制作 {vertical-output: true}

duration = 3.7   # 时长: 5 seconds
framerate = 30  # 频率: 30 Hz

# 配置渲染中显示什么内容
scene_option = mujoco.wrapper.core.MjvOption()
# Visualize the joint axis: 可视化关节轴
scene_option.flags[enums.mjtVisFlag.mjVIS_JOINT] = True

# Simulate and display video.: 进行仿真并显示视频
# 初始化视频数组
frames = []
# Reset state and time: 物理状态和时间初始化
physics.reset()
# 逐帧渲染
while physics.data.time < duration:
  # 物理引擎逐帧运行
  physics.step()
  # 总的画面数目: 10秒 * 30Hz(每秒里面有 30 个图片)
  if len(frames) < physics.data.time * framerate:
    # 获取渲染的当前帧
    pixels = physics.render(scene_option=scene_option)
    # 视频数组中添加当前画面
    frames.append(pixels)
# 播放视频
display_video(frames, framerate)

Note how we collect the video frames. Because physics simulation timesteps are generally much smaller than framerates (the default timestep is 2ms), we don't render after each step.

注意我们是如何收集视频帧的。由于物理仿真的时间步长通常远小于视频的帧率（默认时间步长为 2 毫秒），因此我们不会在每一步仿真后都进行渲染。

## Rendering options: 渲染选项

Like joint visualisation, additional rendering options are exposed as parameters to the `render` method.

与关节可视化类似，`render` 方法还提供了其他渲染选项，可通过参数进行设置。

In [ ]:
#@title Enable transparency and frame visualization: 启用透明效果和坐标系可视化 {vertical-output: true}
# 实例化渲染选项: 用于修改渲染配置
scene_option = mujoco.wrapper.core.MjvOption()
# 显示几何体的局部坐标系
scene_option.frame = enums.mjtFrame.mjFRAME_GEOM
# 设置透明效果
scene_option.flags[enums.mjtVisFlag.mjVIS_TRANSPARENT] = True
# 渲染
pixels = physics.render(scene_option=scene_option)
# 显示
PIL.Image.fromarray(pixels)

In [ ]:
#@title Depth rendering: 深度渲染: 渲染每个像素点到相机的“距离”，即“深度”。 {vertical-output: true}
# 你会得到一张灰度图，图中的亮度代表距离相机的远近——越近越亮，越远越暗。

# 带有画面深度的图形渲染
depth = physics.render(depth=True)
# 将最近的深度设置为参考点
depth -= depth.min()
# 近距离区域的深度进行归一化，增强可视性，防止远距离值压缩图像对比度
depth /= 2*depth[depth <= 1].mean()
# 将深度值限制到 0~1 区间，防止极端值导致图像过曝
pixels = 255*np.clip(depth, 0, 1)
# 图像显示
PIL.Image.fromarray(pixels.astype(np.uint8))

In [ ]:
#@title Segmentation rendering: 分割渲染: 渲染场景中各个物体的“身份”或“类别”。 {vertical-output: true}
# 你会得到一张图，不同物体会以不同的颜色显示。颜色不表示视觉颜色，而是“编号”或“标签”。

# 带有分割图的图形渲染
seg = physics.render(segmentation=True)
# 提取第一个通道，表示每个像素属于哪个对象（以 ID 编号）
geom_ids = seg[:, :, 0]
# 在 MuJoCo 的渲染中，如果一个像素没有对应任何物体（比如背景），其 ID 是 -1
# 加 1 可以避免这些背景像素干扰后续的归一化（变成 0）
geom_ids = geom_ids.astype(np.float64) + 1
# ID 归一化到 0~1 区间，用于灰度图显示
geom_ids = geom_ids / geom_ids.max()
# 把归一化后的灰度值转成 0~255 的范围，变成标准图像格式
pixels = 255*geom_ids
# 图像显示
PIL.Image.fromarray(pixels.astype(np.uint8))

In [ ]:
#@title Projecting from world to camera coordinates: 坐标系转换: 从世界坐标系到相机坐标系 {vertical-output: true}

# 获取红色盒子中心坐标
box_pos = physics.named.data.geom_xpos['red_box']
# 获取红色盒子朝向
box_mat = physics.named.data.geom_xmat['red_box'].reshape(3, 3)
# 获取红色盒子尺寸
box_size = physics.named.model.geom_size['red_box']
# 从中心出发到 8 个角的向量
offsets = np.array([-1, 1]) * box_size[:, None]
# 笛卡尔积生成 8 个角点的局部坐标（相对于中心点）
xyz_local = np.stack(list(itertools.product(*offsets))).T
# 局部坐标通过旋转和平移，转化为全局坐标系
xyz_global = box_pos[:, None] + box_mat @ xyz_local

# 创建齐次坐标形式的角点向量：把 [x, y, z] 扩展成 [x, y, z, 1]，方便相机矩阵投影计算
corners_homogeneous = np.ones((4, xyz_global.shape[1]), dtype=float)
corners_homogeneous[:3, :] = xyz_global

# 创建一个 Camera 实例并获取它的投影矩阵（4×4），用于将世界坐标投影成图像像素坐标
camera = mujoco.Camera(physics)
camera_matrix = camera.matrix

# 相机矩阵 × 齐次坐标 → 得到未归一化的像素位置 (x*s, y*s, s)
xs, ys, s = camera_matrix @ corners_homogeneous
# 进行透视除法，得到最终在图像平面中的 x、y 像素坐标
x = xs / s
y = ys / s

# 渲染相机视图，并叠加投影后的角点坐标
pixels = camera.render()
fig, ax = plt.subplots(1, 1)
# 图像显示
ax.imshow(pixels)
ax.plot(x, y, '*', c='w')
ax.set_axis_off()

# MuJoCo basics and named indexing: MuJoCo 基础知识与命名索引

| 属性         | `mjModel`（模型结构）                        | `mjData`（模拟数据）                         |
|--------------|----------------------------------------------|----------------------------------------------|
| 中文名       | 模型结构                                     | 模拟数据                                     |
| 类型         | 静态，只读                                   | 动态，随仿真变化                             |
| 作用         | 存储模型定义（结构、几何、参数等）           | 存储当前仿真状态（位置、速度、力等）         |
| 生命周期     | 加载 XML 时创建，一般不变                    | 每步仿真更新，实时变化                       |
| 来源         | 来自 MJCF / XML 文件                         | 由 `mjModel` 推导初始化                      |
| 是否可修改   | 一般不建议修改（用于定义结构）               | 可通过接口更新（如 `qpos`、`ctrl` 等）        |
| 常见字段     | `nq`, `nv`, `geom_size`, `body_mass` 等      | `qpos`, `qvel`, `xpos`, `ctrl`, `sensordata` 等 |
| 示例用途     | 获取关节/几何体数量、初始参数等              | 控制机器人动作、读取当前状态、做物理演算     |


## `mjModel`
MuJoCo's `mjModel`, encapsulated in `physics.model`, contains the *model description*, including the default initial state and other fixed quantities which are not a function of the state, e.g. the positions of geoms in the frame of their parent body. The (x, y, z) offsets of the `box` and `sphere` geoms, relative their parent body `box_and_sphere` are given by `model.geom_pos`:

MuJoCo 的 `mjModel`（在代码中通过 `physics.model` 访问）包含*模型的描述信息*，包括默认的初始状态以及其他不依赖于系统状态的固定量，例如：几何体（geoms）在其父 body 坐标系下的位置。

例如，`box` 和 `sphere` 这两个几何体相对于它们的父 `body`（名为 `box_and_sphere`）的 (x, y, z) 偏移量，可以通过 `model.geom_pos` 获取。

In [ ]:
# 结果包含物体位置信息
# 红色盒子: [0. , 0. , 0. ]
# 绿色小球: [0.2, 0.2, 0.2]
# 蓝色方块: [0. , 0.2, 0.2]
# geom 只有几何体, 不包含 joint 铰链信息
physics.model.geom_pos

In [ ]:
# 获取 joint 铰链信息
physics.model.joint('swing')

The `model.opt` structure contains global quantities like: model.opt 结构体包含一些全局参数，例如

In [ ]:
# 渲染单步时长
print('timestep', physics.model.opt.timestep)
# 重力参数 g
print('gravity', physics.model.opt.gravity)

## `mjData`
`mjData`, encapsulated in `physics.data`, contains the *state* and quantities that depend on it. The state is made up of time, generalized positions and generalised velocities. These are respectively `data.time`, `data.qpos` and `data.qvel`.

Let's print the state of the swinging body where we left it:

`mjData`（在代码中通过 `physics.data` 访问）包含系统的状态以及依赖于该状态的各种量。状态由以下部分组成：

时间（data.time）
广义位置（data.qpos）
广义速度（data.qvel）

下面我们来打印一下之前暂停时摆动物体（swinging body）的状态：

In [ ]:
# 当前时间
print('time', physics.data.time)
# 关节角度
print('joint position', physics.data.qpos)
# 关节速度
print('joint velocity', physics.data.qvel)

`physics.data` also contains functions of the state, for example the cartesian positions of objects in the world frame. The (x, y, z) positions of our two geoms are in `data.geom_xpos`:

`physics.data` 还包含一些状态的函数（即由状态计算得到的量），例如物体在世界坐标系中的笛卡尔坐标位置。
我们两个几何体（geoms）在世界坐标系下的 (x, y, z) 位置可以通过 `data.geom_xpos` 获取：

In [ ]:
# 结果包含物体位置信息: 运动过程中的实时数据
# 红色盒子: [ 0.03732699  0.01000174 -0.07097884]
# 绿色小球: [ 0.34785906  0.09320855  0.05804233]
# 蓝色方块: [ 0.17837146  0.19420465  0.09081036]
print(physics.data.geom_xpos)

## Named indexing: 命名索引: 根据物品名称来获取数据

The semantics of the above arrays are made clearer using the `named` wrapper, which assigns names to rows and type names to columns. 使用 named 包装器可以更清晰地表达上述数组的语义，它为行赋予名称，为列赋予类型名称。

In [ ]:
# geom 几何体的实时位置数据
print(physics.named.data.geom_xpos)

Note how `model.geom_pos` and `data.geom_xpos` have similar semantics but very different meanings.
注意，`model.geom_pos` 和 `data.geom_xpos` 在语义上相似，但含义却大不相同。

In [ ]:
# geom 几何体的初始位置数据
print(physics.named.model.geom_pos)

Name strings can be used to index **into** the relevant quantities, making code much more readable and robust.
名称字符串可以用于索引相关的量，使代码更易读且更健壮。

In [ ]:
# 单一物品某个方向上的位置: 绿色球在 z 方向上的坐标
physics.named.data.geom_xpos['green_sphere', 'z']

Joint names can be used to index into quantities in configuration space (beginning with the letter `q`):
关节名称可以用于索引配置空间中的量（以字母 q 开头的量）：

In [ ]:
# 铰链的关节角度
physics.named.data.qpos['swing']

We can mix NumPy slicing operations with named indexing. As an example, we can set the color of the box using its name (`"red_box"`) as an index into the rows of the `geom_rgba` array.

我们可以将 NumPy 的切片操作与命名索引结合使用。例如，我们可以使用物体的名称（如 `"red_box"`）作为 `geom_rgba` 数组行的索引，来设置该盒子的颜色。

In [ ]:
#@title Changing colors using named indexing: 使用命名索引更改颜色(这里修改的不是 data 而是 model) {vertical-output: true}
# 生成一个颜色
random_rgb = np.random.rand(3)
# 红色盒子颜色修改
physics.named.model.geom_rgba['red_box', :3] = random_rgb
# 图形渲染
pixels = physics.render()
# 显示
PIL.Image.fromarray(pixels)

Note that while `physics.model` quantities will not be changed by the engine, we can change them ourselves between steps. This however is generally not recommended, the preferred approach being to modify the model at the XML level using the PyMJCF library, see below.
请注意，虽然引擎不会更改 `physics.model` 中的量，但我们可以在仿真步之间自行修改它们。不过，通常不推荐这样做，更推荐的方法是使用 PyMJCF 库在 XML 层级上修改模型，详见下文。

## Setting the state with `reset_context()`: 使用 `reset_context()` 设置状态

In order for `data` quantities that are functions of the state to be in sync with the state, MuJoCo's `mj_step1()` needs to be called. This is facilitated by the `reset_context()` context, please see in-depth discussion in Section 2.1 of the [tech report](https://arxiv.org/abs/2006.12983).

为了使作为状态函数的 `data` 中的量与状态保持同步，需要调用 MuJoCo 的 `mj_step1()`。这可以通过 `reset_context()` 上下文来实现，详细讨论请参见技术报告第 2.1 节。

In [ ]:
# 角度赋值, 但未更新
physics.named.data.qpos['swing'] = np.pi / 3
# 显示更新后的数据, 仍然保持之前的数据
print('Without reset_context, spatial positions are not updated:',
      physics.named.data.geom_xpos['green_sphere', ['z']])
# 图形渲染
pixels = physics.render()
# 图像显示
PIL.Image.fromarray(pixels.astype(np.uint8))

In [ ]:
# 更新数据
with physics.reset_context():
  # 角度赋值
  physics.named.data.qpos['swing'] = np.pi / 3
# 显示更新后的数据, 将变得不一样
print('After reset_context, positions are up-to-date:',
      physics.named.data.geom_xpos['green_sphere', ['z']])
# 图形渲染
pixels = physics.render()
# 图像显示
PIL.Image.fromarray(pixels.astype(np.uint8))

## Free bodies: the self-inverting "tippe-top": 自由刚体：自翻转的“陀螺顶”（tippe-top）

A free body is a body with a `free` joint, with 6 movement DOFs: 3 translations and 3 rotations. We could give our `box_and_sphere` body a free joint and watch it fall, but let's look at something more interesting. A "tippe top" is a spinning toy which flips itself on its head ([Wikipedia](https://en.wikipedia.org/wiki/Tippe_top)). We model it as follows:

自由刚体是带有自由关节的刚体，具有 6 个运动自由度：3 个平移自由度和 3 个旋转自由度。我们本可以为 `box_and_sphere` 刚体添加一个自由关节并观察其下落过程，但让我们来看一个更有趣的例子。

“tippe top”（翻转陀螺）是一种旋转玩具，它在旋转时会自己翻转过来（见维基百科）。我们对它进行如下建模：

In [ ]:
#@title The "tippe-top" model{vertical-output: true}


tippe_top = """
<mujoco model="tippe top">
  <option integrator="RK4"/>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1="0.9375 0.7226 0.04296"
     rgb2="0 0 0" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>
  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <freejoint/>
      <geom name="ball" type="sphere" size=".02" rgba=".9 .9 .9 1"/>
      <geom name="stem" type="cylinder" pos="0 0 .02" size="0.004 .008"/>
      <geom name="ballast" type="box" size=".023 .023 0.005"  pos="0 0 -.015"
       contype="0" conaffinity="0" group="3"/>
    </body>
  </worldbody>
  <keyframe>
    <key name="spinning" qpos="0 0 0.02 1 0 0 0" qvel="0 0 0 0 1 50" />
  </keyframe>
</mujoco>
"""
# 导入模型
physics = mujoco.Physics.from_xml_string(tippe_top)
# 设置相机并渲染
PIL.Image.fromarray(physics.render(camera_id='closeup'))

Note several new features of this model definition:
0. The free joint is added with the `<freejoint/>` clause, which is similar to `<joint type="free"/>`, but prohibits unphysical attributes like friction or stiffness.
1. We use the `<option/>` clause to set the integrator to the more accurate Runge Kutta 4th order.
2. We define the floor's grid material inside the `<asset/>` clause and reference it in the floor geom.
3. We use an invisible and non-colliding box geom called `ballast` to move the top's center-of-mass lower. Having a low center of mass is  (counter-intuitively) required for the flipping behaviour to occur.
4. We save our initial spinning state as a keyframe. It has a high rotational velocity around the z-axis, but is not perfectly oriented with the world.
5. We define a `<camera>` in our model, and then render from it using the `camera_id` argument to `render()`.
Let us examine the state:

请注意此模型定义中的几个新特性：

0. 使用 `<freejoint/>` 子句添加自由关节，它类似于 `<joint type="free"/>`，但禁止使用不符合物理规律的属性（如摩擦或刚度）。

1. 使用 `<option/>` 子句将积分器设置为更精确的四阶 Runge-Kutta 方法。

2. 在 `<asset/>` 子句中定义地板使用的网格材质，并在地板的几何体中引用它。

3. 使用一个不可见且不会发生碰撞的 box 几何体，命名为 `ballast`，用来将陀螺的质心向下移动。让质心偏低（虽然听起来违反直觉）是实现翻转行为所必需的。

4. 将初始旋转状态保存为一个关键帧（keyframe）。该状态在 z 轴上具有较高的旋转速度，但在方向上与世界坐标系略有偏差。

5. 在模型中定义了一个 `<camera>`，然后通过 `render()` 函数的 `camera_id` 参数使用该相机进行渲染。

下面我们来检查这个状态：

In [ ]:
# 自由体有 6 个自由度（DOF）：3 个平移、3 个旋转
# 位置根据自由度由 [x, y, z] 和四元数(表示方向) [qw, qx, qy, qz] 表示
# 四元数比欧拉角更稳定、无奇点，适合模拟旋转
print('positions', physics.data.qpos)
# 角速度由 x y z 和球坐标表示
print('velocities', physics.data.qvel)

The velocities are easy to interpret, 6 zeros, one for each DOF. What about the length-7 positions? We can see the initial 2cm height of the body; the subsequent four numbers are the 3D orientation, defined by a *unit quaternion*. These normalized four-vectors, which preserve the topology of the orientation group, are the reason that `data.qpos` can be bigger than `data.qvel`: 3D orientations are represented with **4** numbers while angular velocities are **3** numbers.

速度很容易理解——6 个零，分别对应每个自由度的速度。而长度为 7 的位置向量又代表什么呢？我们可以看到刚体初始高度为 2 厘米；接下来的四个数字表示 3D 方向，由一个单位四元数定义。

这种归一化的四维向量能够保持方向空间的拓扑结构，因此是 data.qpos 可能比 data.qvel 更长的原因之一：

三维方向使用 4 个数字（四元数）表示

而角速度只需要 3 个数字即可表示。

In [ ]:
#@title Video of the tippe-top: 倒立陀螺的视频 {vertical-output: true}

# 视频时间
duration = 6
# 渲染帧率
framerate = 120

# 画面帧列表
frames = []
# 初始状态: 指向关键帧 0, 加载 model 中预设的状态
physics.reset(0)
# 到指定时间前渲染
while physics.data.time < duration:
  # 逐帧渲染
  physics.step()
  # 渲染帧数
  if len(frames) < (physics.data.time) * framerate:
    # 根据相机视角渲染
    pixels = physics.render(camera_id='closeup')
    # 增加画面
    frames.append(pixels)
# 播放视频
display_video(frames, framerate)

### Measuring values from `physics.data`: 从 `physics.data` 中获取数值
The `physics.data` structure contains all of the dynamic variables and intermediate results produced by the simulation. These are expected to change on each timestep.

Below we simulate for 2000 timesteps and plot the state and height of the sphere as a function of time.


`physics.data` 结构包含仿真过程中产生的所有动态变量和中间结果。这些值在每一个时间步都会发生变化。

下面我们将仿真运行 2000 个时间步，并绘制球体的状态和高度随时间变化的图像。

In [ ]:
#@title Measuring values {vertical-output: true}

# 时间戳
timevals = []
# 陀螺旋转部分的角速度（3维向量）
angular_velocity = []
# 陀螺杆顶（stem）的高度（z 坐标）
stem_height = []

# 恢复初始状态
physics.reset(0)
# 开始运行
while physics.data.time < duration:
  # 逐帧模拟
  physics.step()
  # 获取当前时间戳
  timevals.append(physics.data.time)
  # 记录陀螺的角速度部分（自由体的 qvel[3:6] 是 [wx, wy, wz]）
  angular_velocity.append(physics.data.qvel[3:6].copy())
  # 记录 stem（杆子）这个几何体在世界坐标中的 z 坐标（高度）
  stem_height.append(physics.named.data.geom_xpos['stem', 'z'])

# 图像清晰度
dpi = 100
# 宽度
width = 480
# 高度
height = 640
# 计算出 Matplotlib 所需的英寸尺寸
figsize = (width / dpi, height / dpi)
# 创建一个 2 行 1 列的子图布局，sharex=True 表示两个图共用 x 轴（时间）
_, ax = plt.subplots(2, 1, figsize=figsize, dpi=dpi, sharex=True)

# 绘制角速度，y 值是一个三维向量，会绘制 3 条曲线（wx, wy, wz）
ax[0].plot(timevals, angular_velocity)
# 标题设置
ax[0].set_title('angular velocity')
# y 轴标签
ax[0].set_ylabel('radians / second')

# 绘制第二张图
ax[1].plot(timevals, stem_height)
# 第二个图的 x 轴标签
ax[1].set_xlabel('time (seconds)')
# 第二个图的 y 轴标签
ax[1].set_ylabel('meters')
# 第二个图的标题
_ = ax[1].set_title('stem height')

# PyMJCF tutorial: 用 python 画模型, 避免直接写 xml




This library provides a Python object model for MuJoCo's XML-based
[MJCF](http://www.mujoco.org/book/modeling.html) physics modeling language.
- 这个库为 MuJoCo 的 MJCF 物理建模语言（基于 XML）提供了一个 Python 对象模型。换句话说，它让你可以用 Python 来构建、修改和操作 MJCF 模型，而不必直接写 XML。


The goal of the library is to allow users to easily interact with and modify MJCF models in Python, similarly to what the JavaScript DOM does for HTML.
- 这个库的目标是让用户能够像操作 JavaScript 的 DOM 那样，在 Python 中方便地交互和修改 MJCF 模型。也就是说，像操作网页结构那样操作模型结构。


A key feature of this library is the ability to easily compose multiple separate MJCF models into a larger one. Disambiguation of duplicated names from different models, or multiple instances of the same model, is handled automatically.
- 这个库的一个重要功能是：可以轻松地将多个独立的 MJCF 模型组合成一个更大的整体模型。
- 如果多个模型中有重复命名的问题（例如多个模型里都叫 joint1），或者你导入了同一个模型的多个实例，库会自动处理这些命名冲突，避免混乱。


One typical use case is when we want robots with a variable number of joints. This is a fundamental change to the kinematics, requiring a new XML descriptor and new binary model to be compiled.
- 一个典型的使用场景是：当我们需要创建关节数可变的机器人时。
- 因为关节数量的变化会从根本上改变机器人的运动学结构，所以需要重新生成 XML 描述文件，并重新编译成 MuJoCo 使用的二进制模型。


The following snippets realise this scenario and provide a quick example of this library's use case.
- 接下来的代码片段展示了这种场景的实现，并快速演示了该库的典型用法。

In [ ]:
class Leg(object):
  """
  定义一个名为 Leg 的类，表示一个拥有两个自由度（2-DoF，两个关节）的机器人腿部。
  每个关节都由一个位置控制器（actuator）驱动。
  """
  def __init__(self, length, rgba):
    # 构造函数，参数：
    # - length：腿段的长度
    # - rgba：颜色，用于可视化（红绿蓝透明度）

    # 创建 MJCF 模型的根元素（即 MJCF 文件的顶层 <mujoco> 元素）
    self.model = mjcf.RootElement()

    # 设置默认属性（默认 joint 和 geom 的属性），这会影响后续创建的 joint 和 geom：
    # 所有关节默认具有阻尼 2，有助于稳定模拟
    self.model.default.joint.damping = 2
    # 所有关节默认是铰链型，只允许绕一个轴旋转
    self.model.default.joint.type = 'hinge'
    # 所有几何体默认是胶囊体（用于表示肢体）
    self.model.default.geom.type = 'capsule'
    # 设置所有几何体的默认颜色为传入的 rgba（可用于区分多个腿）
    self.model.default.geom.rgba = rgba  # Continued below...

    # 创建大腿（thigh）部分：
    # 向世界坐标系添加一个身体（即大腿部分）
    self.thigh = self.model.worldbody.add('body')
    # 在大腿上添加一个绕 z 轴旋转的铰链关节，表示髋关节（hip）
    self.hip = self.thigh.add('joint', axis=[0, 0, 1])
    # 添加一个从原点延伸到 (length, 0, 0) 的胶囊体，用于表示大腿的形状和碰撞体积
    self.thigh.add('geom', fromto=[0, 0, 0, length, 0, 0], size=[length/4])

    # 创建小腿（shin）部分：
    # 在大腿的末端添加一个新的 body，表示小腿，位置在大腿的末端
    self.shin = self.thigh.add('body', pos=[length, 0, 0])
    # 在小腿上添加一个绕 y 轴旋转的铰链关节，表示膝关节（knee）
    self.knee = self.shin.add('joint', axis=[0, 1, 0])
    # 添加一个向下延伸的小腿胶囊体，从当前位置 (0,0,0) 向 z 轴负方向延伸
    self.shin.add('geom', fromto=[0, 0, 0, 0, 0, -length], size=[length/5])

    # 添加两个位置控制器（actuator）：
    # 为髋关节添加一个位置控制器，控制器的刚度系数 kp=10
    self.model.actuator.add('position', joint=self.hip, kp=10)
    # 为膝关节添加一个位置控制器，控制器的刚度系数 kp=10
    self.model.actuator.add('position', joint=self.knee, kp=10)

The `Leg` class describes an abstract articulated leg, with two joints and corresponding proportional-derivative actuators.

Note that:

- MJCF attributes correspond directly to arguments of the `add()` method.
- When referencing elements, e.g when specifying the joint to which an actuator is attached, the MJCF element itself is used, rather than the name string.

`Leg` 类描述了一个抽象的关节腿结构，包含两个关节和对应的比例-微分（PD）控制器。

请注意：
- MJCF 中的属性直接对应于 `add()` 方法的参数。
- 在引用元素时（例如指定 actuator 所连接的关节），使用的是 MJCF 元素对象本身，而不是元素的名称字符串。

In [ ]:
# 定义躯干的半径为 0.1 米（单位通常是米）
BODY_RADIUS = 0.1
# 躯干的三维尺寸，使用椭球体来表示，其长宽为半径，高度为半径的一半
BODY_SIZE = (BODY_RADIUS, BODY_RADIUS, BODY_RADIUS / 2)
# 创建一个固定种子的随机数生成器，用于生成可重复的随机颜色（RGBA）
random_state = np.random.RandomState(726)

def make_creature(num_legs):
  """ 定义一个函数，用于构造一个具有指定数量腿 "num_legs" 的生物模型"""
  # 生成一个随机的 RGBA 颜色值，其中颜色是随机的，但 alpha（透明度）始终为 1（不透明）
  rgba = random_state.uniform([0, 0, 0, 1], [1, 1, 1, 1])
  # 创建 MJCF 的根节点，表示一个新的模拟模型
  model = mjcf.RootElement()
   # 设置角度单位为弧度（MuJoCo 默认支持角度或弧度）
  model.compiler.angle = 'radian'

  # 添加一个名为 'torso' 的几何体，类型为椭球体，尺寸为 BODY_SIZE，用于表示生物的躯干
  model.worldbody.add(
      'geom', name='torso', type='ellipsoid', size=BODY_SIZE, rgba=rgba)

  # 遍历每一条腿，将它们等距放置在躯干周围
  for i in range(num_legs):
    # 计算当前腿的角度 theta，使腿均匀分布在一个圆上
    theta = 2 * i * np.pi / num_legs
    # 根据角度计算髋关节在 xy 平面上的位置，z 保持为 0
    hip_pos = BODY_RADIUS * np.array([np.cos(theta), np.sin(theta), 0])
    # 在躯干表面添加一个 site（定位点），并设置其朝向为 theta 方向（用于旋转腿）
    hip_site = model.worldbody.add('site', pos=hip_pos, euler=[0, 0, theta])
    # 创建一个 Leg 实例（两关节的机械腿），长度为躯干半径，颜色与躯干一致
    leg = Leg(length=BODY_RADIUS, rgba=rgba)
    # 输出 leg 信息
    # print('leg info:', leg.model)
    # 将腿模型附加到 site 上，site 的姿态会影响整个腿的位置和朝向
    hip_site.attach(leg.model)
  # 返回完整的 MJCF 模型对象，包含一个躯干和多个附着的腿
  return model

The `make_creature` function uses PyMJCF's `attach()` method to procedurally attach legs to the torso. Note that at this stage both the torso and hip attachment sites are children of the `worldbody`, since their parent body has yet to be instantiated. We'll now make an arena with a chequered floor and two lights, and place our creatures in a grid.

- `make_creature` 函数使用了 PyMJCF 的 `attach()` 方法，以程序化方式将腿连接到躯干上。需要注意的是，在这个阶段，躯干和髋关节的附着点（site）都是 `worldbody` 的子元素，因为它们的父 body 尚未被创建。

- 接下来，我们将构建一个竞技场（arena），其地面为棋盘格图案，并添加两个光源，然后将多个生物模型按网格排列放置在场地中。

In [ ]:
#@title Six Creatures on a floor: 六只生物站在地板上{vertical-output: true}

# 创建一个新的 MJCF 模型，作为“竞技场”（arena）的根元素
arena = mjcf.RootElement()
# 棋盘格纹理（checker），设置为 2D 贴图，尺寸为 300x300
chequered = arena.asset.add('texture', type='2d', builtin='checker', width=300,
                            height=300, rgb1="0.9375 0.7226 0.04296", rgb2="0 0 0")
# 创建一个材质，使用上面的棋盘格纹理，并设置为在地面上重复 5x5 次，具有一定反射率
grid = arena.asset.add('material', name='grid', texture=chequered,
                       texrepeat=[5, 5], reflectance=.2)
# 在世界中添加一个平面几何体作为地板，尺寸为 2x2（单位通常为米），使用棋盘格材质
arena.worldbody.add('geom', type='plane', size=[2, 2, .1], material=grid)
# 在地板两侧添加两个光源，分别位于左边和右边，指向地板中心，模拟自然光照效果
for x in [-2, 2]:
  arena.worldbody.add('light', pos=[x, -1, 3], dir=[-x, 1, -2])

# 根据 legs 范围创造生物体
creatures = [make_creature(num_legs=num_legs) for num_legs in range(0, 8)]

# 将它们按网格方式放置在竞技场中
# 生物的初始高度
height = .15
# 网格间距
grid = 12 * BODY_RADIUS
# 等距离摆放生物
xpos, ypos, zpos = np.meshgrid([-grid, -grid/2, 0, grid/2, grid], [0, grid], [height])

# 逐个添加生物体
for i, model in enumerate(creatures):
  # 网格上的每个位置作为生物的出生点
  spawn_pos = (xpos.flat[i], ypos.flat[i], zpos.flat[i])
  # 创建 site 以定位每只生物的起始位置
  spawn_site = arena.worldbody.add('site', pos=spawn_pos, group=3)
  # 将生物模型附加到 site 上，并添加一个 freejoint 使其在空间中可自由移动（6 自由度）
  spawn_site.attach(model).add('freejoint')

# 配置渲染
physics = mjcf.Physics.from_mjcf_model(arena)
# 图像显示
PIL.Image.fromarray(physics.render())

Multi-legged creatures, ready to roam! Let's inject some controls and watch them move. We'll generate a sinusoidal open-loop control signal of fixed frequency and random phase, recording both video frames and the horizontal positions of the torso geoms, in order to plot the movement trajectories.

- 多足生物，准备出发啦！
- 现在让我们加入一些控制信号，看看它们如何移动。

- 我们将为每只生物生成一个固定频率、随机初始相位的正弦波开环控制信号，同时记录视频帧和躯干几何体在水平方向的位置，以便后续绘制它们的运动轨迹。

In [ ]:
#@title Video of the movement: 开始运动{vertical-output: true}

# 运动时间
duration = 10   # (Seconds)
# 帧率
framerate = 120  # (Hz)
# 视频列表
video = []
# 横坐标列表
pos_x = []
# 纵坐标列表
pos_y = []
# 躯干几何体列表
torsos = []  # List of torso geom elements.
# 驱动器列表
actuators = []  # List of actuator elements.
# 对每个生物体进行遍历
for creature in creatures:
  # 找到名为 'torso' 的几何体并加入 torsos 列表
  torsos.append(creature.find('geom', 'torso'))
  # 找到该生物中所有的执行器（如髋关节、膝关节）并加入 actuators 列表
  actuators.extend(creature.find_all('actuator'))

# 控制信号的频率、相位和幅值
# 控制信号的频率为 5Hz 即 1 秒钟震荡 5 次
freq = 10
# 为每个执行器生成一个 0 ~ 2π 范围内的随机相位，确保它们不同步，动作更自然
phase = 2 * np.pi * random_state.rand(len(actuators))
# 控制信号的幅度（最大控制量）设置为 0.9
amp = 0.9

# 重置物理仿真状态，准备开始新的模拟
physics.reset()
# 没达到设定的 duration（例如 3 秒、5 秒）时，持续模拟
while physics.data.time < duration:
  # 给所有执行器注入控制信号，然后推进一步仿真
  physics.bind(actuators).ctrl = amp * np.sin(freq * physics.data.time + phase)
  # 推进物理模拟一步（更新模型状态）
  physics.step()

  # 保存每个生物躯干的横向位置 x 坐标
  pos_x.append(physics.bind(torsos).xpos[:, 0].copy())
  # 保存每个生物躯干的横向位置 y 坐标
  pos_y.append(physics.bind(torsos).xpos[:, 1].copy())

  # 按帧率保存视频帧
  if len(video) < physics.data.time * framerate:
    # 逐帧渲染
    pixels = physics.render()
    # 添加到视频
    video.append(pixels.copy())

# 播放视频
display_video(video, framerate)

In [ ]:
#@title Movement trajectories: 运动轨迹{vertical-output: true}

# 颜色
creature_colors = physics.bind(torsos).rgba[:, :3]
# 图片尺寸
fig, ax = plt.subplots(figsize=(4, 4))
# 设置颜色
ax.set_prop_cycle(color=creature_colors)
# 显示图片
_ = ax.plot(pos_x, pos_y, linewidth=4)

The plot above shows the corresponding movement trajectories of creature positions. Note how `physics.bind(torsos)` was used to access both `xpos` and `rgba` values. Once the `Physics` had been instantiated by `from_mjcf_model()`, the `bind()` method will expose both the associated `mjData` and `mjModel` fields of an `mjcf` element, providing unified access to all quantities in the simulation.

- 上图展示了生物位置对应的运动轨迹。请注意，我们是通过 `physics.bind(torsos)` 来访问 `xpos` 和 `rgba` 的值的。

- 一旦使用 `from_mjcf_model()` 实例化了 Physics 对象，`bind()` 方法就可以同时访问某个 MJCF 元素所关联的 `mjData` 和 `mjModel` 字段，从而统一地获取该元素在模拟中的所有相关属性。

# Composer tutorial: 实体化教程

In this tutorial we will create a task requiring our "creature" above to press a colour-changing button on the floor with a prescribed force. We begin by implementing our creature as a `composer.Entity`:

- MuJoCo 负责底层的物理仿真，composer 负责高层的“任务拼装”与逻辑管理
- 模块化构建复杂环境（Entity + Arena + Task）
- 在本教程中，我们将创建一个任务，要求上面构建的“生物”用指定的力按下地板上的一个变色按钮。
我们首先将这个生物实现为一个 `composer.Entity` 实体。

In [ ]:
#@title The `Creature` class: 生物体类


class Creature(composer.Entity):
  """定义一个 Creature 类，继承自 `composer.Entity`，用于构造多足生物作为一个模块化实体"""
  # _build 是 Entity 类中必须实现的方法，用于构建 MJCF 模型
  def _build(self, num_legs):
    # 调用之前定义的 make_creature() 函数，创建一个生物模型并保存为 _model
    self._model = make_creature(num_legs)

  # 定义观测器（Observables）, 用于后续任务和训练中收集传感器数据
  def _build_observables(self):
    return CreatureObservables(self)

  @property
  # 提供 mjcf_model 属性接口，返回 MJCF 根模型
  def mjcf_model(self):
    return self._model

  @property
  # 提供 actuators 属性接口，返回所有执行器组成的元组
  def actuators(self):
    return tuple(self._model.find_all('actuator'))


# 定义一个 CreatureObservables 类，继承自 composer.Observables
# 用于提供生物的可观测特征（如关节角度、关节速度）
class CreatureObservables(composer.Observables):
  # 定义 joint_positions 观测项，返回所有关节的位置（qpos）
  @composer.observable
  def joint_positions(self):
    # 查找所有关节
    all_joints = self._entity.mjcf_model.find_all('joint')
    # 创建一个用于获取 qpos 的特征
    return observable.MJCFFeature('qpos', all_joints)

  # 定义 joint_velocities 观测项，返回所有关节的速度（qvel）
  @composer.observable
  def joint_velocities(self):
    # 查找所有关节
    all_joints = self._entity.mjcf_model.find_all('joint')
    # 创建一个用于获取 qvel 的特征
    return observable.MJCFFeature('qvel', all_joints)

The `Creature` Entity includes generic Observables for joint angles and velocities. Because `find_all()` is called on the `Creature`'s MJCF model, it will only return the creature's leg joints, and not the "free" joint with which it will be attached to the world. Note that Composer Entities should override the `_build` and `_build_observables` methods rather than `__init__`. The implementation of `__init__` in the base class calls `_build` and `_build_observables`, in that order, to ensure that the entity's MJCF model is created before its observables. This was a design choice which allows the user to refer to an observable as an attribute (`entity.observables.foo`) while still making it clear which attributes are observables. The stateful `Button` class derives from `composer.Entity` and implements the `initialize_episode` and `after_substep` callbacks.

- Creature 实体包含了用于关节角度和速度的通用观测项（Observables）。
- 由于 `find_all()` 是在 Creature 的 MJCF 模型上调用的，它只会返回生物腿部的关节，而不会返回用于将生物连接到世界的 "free" 关节。

- 需要注意的是，Composer 实体应当重写 `_build` 和 `_build_observables` 方法，而不是 `__init__` 方法。
- 基类中的 `__init__` 实现会依次调用 `_build` 和 `_build_observables`，从而确保在创建观测项之前，实体的 MJCF 模型已经构建完成。

- 这是一个有意为之的设计选择，它允许用户以属性的形式访问观测项（例如 `entity.observables.foo`），同时又能清晰地区分哪些属性是观测项。

- 另外，带有状态的 Button 类继承自 `composer.Entity`，并实现了 `initialize_episode` 和 `after_substep` 回调方法。

In [ ]:
#@title The `Button` class: 变色按钮定义

# 这个 Button 类创建了一个带有触觉传感器的按钮
# 当外部施加的力在一定范围内，它会变绿，表示“被按下”
# 并记录被按下的时间步数。可以用于任务奖励、交互等用途

# 为了计算接触力时进行时间平均，表示在每个控制时间步内，物理模拟器运行了 25 次子步
NUM_SUBSTEPS = 25


class Button(composer.Entity):
  """一个按钮实体，被按下时如果作用力在某个范围内会改变颜色"""

  # _build() 是 dm_control Composer 实体的初始化钩子，在构建组件时调用
  # target_force_range 是按钮被认为被“激活”所需的力的范围
  def _build(self, target_force_range=(5, 10)):
    # 解析最小/最大力
    self._min_force, self._max_force = target_force_range
    # 创建一个 MJCF 模型的根节点
    self._mjcf_model = mjcf.RootElement()
    # 添加一个圆柱体几何体（即按钮本体），红色 ([1, 0, 0, 1])，尺寸为半径 0.25，高度 0.02
    self._geom = self._mjcf_model.worldbody.add(
        'geom', type='cylinder', size=[0.25, 0.02], rgba=[1, 0, 0, 1])
    # 添加一个透明的 site，用于放置传感器
    # 它的尺寸稍大，用来确保传感器覆盖整个按钮表面
    self._site = self._mjcf_model.worldbody.add(
        'site', type='cylinder', size=self._geom.size*1.01, rgba=[1, 0, 0, 0])
    # 添加一个触觉传感器，作用于该 site，检测与之接触的力
    self._sensor = self._mjcf_model.sensor.add('touch', site=self._site)
    # 记录按钮被激活的时间步数（用于奖励等逻辑）
    self._num_activated_steps = 0

  # 返回这个按钮的观测定义，关联下面的 ButtonObservables 类
  def _build_observables(self):
    return ButtonObservables(self)

  # 暴露模型给上层使用，比如加入环境中
  @property
  def mjcf_model(self):
    return self._mjcf_model
  # 根据当前触觉传感器的力判断按钮是否被激活，并改变颜色
  def _update_activation(self, physics):
    # 获取当前子步的触觉传感器读数
    current_force = physics.bind(self.touch_sensor).sensordata[0]
    # 判断压力是否在指定激活范围内
    self._is_activated = (current_force >= self._min_force and
                          current_force <= self._max_force)
    # 改变颜色：绿色表示激活，红色表示未激活
    physics.bind(self._geom).rgba = (
        [0, 1, 0, 1] if self._is_activated else [1, 0, 0, 1])
    # 如果激活就记录一次
    self._num_activated_steps += int(self._is_activated)

  # 每次环境重置时调用
  def initialize_episode(self, physics, random_state):
    # 初始化奖励
    self._reward = 0.0
    # 初始化激活次数
    self._num_activated_steps = 0
    # 开始更新传感器状态
    self._update_activation(physics)

  # 在每个子步后调用，持续检查激活状态并更新颜色
  def after_substep(self, physics, random_state):
    self._update_activation(physics)

  # 接触传感器
  @property
  def touch_sensor(self):
    return self._sensor

  # 外部访问激活次数
  @property
  def num_activated_steps(self):
    return self._num_activated_steps


class ButtonObservables(composer.Observables):
  """将按钮的状态暴露给观察空间"""
  # 观测传感器状态
  @composer.observable
  def touch_force(self):
    # 使用 observable.MJCFFeature 从传感器获取数据
    # buffer_size=NUM_SUBSTEPS 表示使用过去 25 个子步的数据
    # aggregator='mean' 表示对这些值求平均
    return observable.MJCFFeature('sensordata', self._entity.touch_sensor,
                                  buffer_size=NUM_SUBSTEPS, aggregator='mean')

Note how the Button counts the number of sub-steps during which it is pressed with the desired force. It also exposes an `Observable` of the force being applied to the button, whose value is an average of the readings over the physics time-steps.

We import some `variation` modules and an arena factory:

注意，Button 会统计在作用力满足条件的情况下，被按下的物理子步数。同时它还提供了一个 `Observable`（可观测量），用于观察施加在按钮上的力，该值是多个物理时间步内传感器读数的平均值。

我们还导入了一些变化模块`variation` modules 和一个竞技场工厂（arena factory）：

In [ ]:
#@title Random initialiser using `composer.variation`: 随机初始化器 UniformCircle

class UniformCircle(variation.Variation):
  """在一个半径为 `distance` 的圆周上均匀采样一个水平位置。"""
  def __init__(self, distance):
    # distance：圆的半径，可以是固定值或可变分布
    self._distance = distance
    # heading：角度（朝向），在 [0, 2π] 之间均匀分布，代表从圆心出发的方向
    self._heading = distributions.Uniform(0, 2*np.pi)

  def __call__(self, initial_value=None, current_value=None, random_state=None):
    # 调用时，利用 variation.evaluate 从当前的 distance 和 heading 中采样
    distance, heading = variation.evaluate(
        (self._distance, self._heading), random_state=random_state)
    # 通过三角函数将角度和半径转为 (x, y, 0) 的位置坐标（z轴为0）
    return (distance*np.cos(heading), distance*np.sin(heading), 0)

In [ ]:
#@title The `PressWithSpecificForce` task: 定义了一个任务类，要求生物按按钮（并控制按的力度）

# 按压按钮的任务
class PressWithSpecificForce(composer.Task):

  def __init__(self, creature):
    # 生物体
    self._creature = creature
    # 地板
    self._arena = floors.Floor()
    # 获取 MJCF 模型
    mjcf_model = self._arena.mjcf_model
    # 生物体加到地板上
    self._arena.add_free_entity(self._creature)
    # 添加灯光
    self._arena.mjcf_model.worldbody.add('light', pos=(0, 0, 4))
    # 按钮
    self._button = Button()
    # 附加到竞技场
    self._arena.attach(self._button)

    # 生物初始位置为原点上方 0.15 米
    self._creature_initial_pose = (0, 0, 0.15)
    # 以 0.5~0.75 米为半径
    button_distance = distributions.Uniform(0.5, .75)
    # 按钮的位置是在圆周上随机选一点
    self._button_initial_pose = UniformCircle(button_distance)

    # 创建 MJCF（模型结构）变化器，用于增加任务多样性
    self._mjcf_variator = variation.MJCFVariator()
    # 创建物理参数变化器，用于增加任务多样性
    self._physics_variator = variation.PhysicsVariator()

    # 加入加性噪声（小的正态分布扰动）。
    pos_corrptor = noises.Additive(distributions.Normal(scale=0.01))
    # 对关节位置添加噪声
    self._creature.observables.joint_positions.corruptor = pos_corrptor
    # 启用这个观测量
    self._creature.observables.joint_positions.enabled = True
    # 对关节速度观测加入乘性噪声, 使得感知系统更加真实
    vel_corruptor = noises.Multiplicative(distributions.LogNormal(sigma=0.01))
    # 对关节速度添加噪声
    self._creature.observables.joint_velocities.corruptor = vel_corruptor
    # 启用这个观测量
    self._creature.observables.joint_velocities.enabled = True
    # 启用按钮的触觉传感器观测，之前在 Button 类中定义
    self._button.observables.touch_force.enabled = True

    def to_button(physics):
      # 将按钮的位置转换为相对于生物的本地坐标（用于更有效的策略学习）
      button_pos, _ = self._button.get_pose(physics)
      return self._creature.global_vector_to_local_frame(physics, button_pos)

    # 存储这个任务中自定义的观测量
    self._task_observables = {}
    #  observable.Generic(...) 创建一个自定义观测量，它的值由 to_button 函数提供
    # to_button(physics) 会返回按钮在生物本体坐标系下的位置，通常是一个 (x, y, z) 向量
    self._task_observables['button_position'] = observable.Generic(to_button)

    # 对每一项调用 obs.enabled = True，表示在仿真中启用它
    # 如果不启用，即使你定义了观测，它也不会出现在 agent 能看到的观测数据里
    for obs in self._task_observables.values():
      obs.enabled = True

    # 控制时间步是物理子步的倍数
    self.control_timestep = NUM_SUBSTEPS * self.physics_timestep

  # 返回根实体（整个场景）
  @property
  def root_entity(self):
    return self._arena

  # 返回任务相关的观测集合
  @property
  def task_observables(self):
    return self._task_observables

  # 应用 MJCF 层级的随机变化
  def initialize_episode_mjcf(self, random_state):
    self._mjcf_variator.apply_variations(random_state)

  # 定义一个函数，用于初始化每一轮仿真（episode）
  def initialize_episode(self, physics, random_state):
    # 使用物理变化器（PhysicsVariator）对物理引擎进行扰动，例如：关节阻尼、质量、摩擦等的微小随机变化
    # 目的是：让每个 episode 稍有不同，增加策略的泛化能力
    self._physics_variator.apply_variations(physics, random_state)
    # 利用 variation.evaluate(...) 根据预设的初始位置/分布生成一组采样值
    # self._creature_initial_pose: 生物初始位置
    # self._button_initial_pose: 按钮初始位置
    creature_pose, button_pose = variation.evaluate(
        (self._creature_initial_pose, self._button_initial_pose),
        random_state=random_state)
    # 将生物的位置设置为 creature_pose
    self._creature.set_pose(physics, position=creature_pose)
    # 将按钮的位置设置为 button_pose
    self._button.set_pose(physics, position=button_pose)

  # 计算奖励分数
  def get_reward(self, physics):
    return self._button.num_activated_steps / NUM_SUBSTEPS

In [ ]:
#@title Instantiating an environment: 环境初始化{vertical-output: true}

# 生物体
creature = Creature(num_legs=6)
# 添加任务
task = PressWithSpecificForce(creature)
# 环境初始状态
env = composer.Environment(task, random_state=np.random.RandomState(42))
# 环境初始化
env.reset()
# 显示图像
PIL.Image.fromarray(env.physics.render())

# The *Control Suite*: 训练和评估强化学习算法的“标准题库”，训练和测试 RL（强化学习）算法

The **Control Suite** is a set of stable, well-tested tasks designed to serve as a benchmark for continuous control learning agents. Tasks are written using the basic MuJoCo wrapper interface. Standardised action, observation and reward structures make suite-wide benchmarking simple and learning curves easy to interpret. Control Suite domains are not meant to be modified, in order to facilitate benchmarking. For full details regarding benchmarking, please refer to our original [publication](https://arxiv.org/abs/1801.00690).
- Control Suite（控制套件） 是一组稳定且经过充分测试的任务，旨在作为连续控制学习算法的基准测试工具。这些任务是基于 MuJoCo 的基础封装接口编写的。标准化的动作、观测和奖励结构使得跨任务的基准评估变得简单，学习曲线也更容易理解。
- 为了便于基准测试，Control Suite 中的任务领域并不建议被修改。

A video of solved benchmark tasks is [available here](https://www.youtube.com/watch?v=rAai4QzcYbs&feature=youtu.be).
- 已经完成的基准任务的视频展示可以在[这里观看](https://www.youtube.com/watch?v=rAai4QzcYbs&feature=youtu.be).


The suite come with convenient module level tuples for iterating over tasks:
- 该套件还提供了便捷的模块级元组（tuples），用于遍历任务列表：



In [ ]:
#@title Iterating over tasks: 遍历任务{vertical-output: true}

# 计算所有任务域名（domain）的最大长度，用于格式化输出时对齐
max_len = max(len(d) for d, _ in suite.BENCHMARKING)
# 遍历 Control Suite 中的基准任务（benchmarking tasks），每个是 (domain, task) 对，比如 ("cheetah", "run")
for domain, task in suite.BENCHMARKING:
  # 打印每个任务
  print(f'{domain:<{max_len}}  {task}')

In [ ]:
#@title Loading and simulating a `suite` task: 加载并模拟一个 `suite` 任务{vertical-output: true}

# 加载环境
# 创建一个随机状态对象，种子设为 42，确保结果可重复
random_state = np.random.RandomState(42)
# 加载 'hopper' 环境中的 'stand' 任务，传入随机状态作为任务参数
env = suite.load('hopper', 'stand', task_kwargs={'random': random_state})

# 使用随机动作模拟一个回合
# 设置模拟时长为 4 秒
duration = 4
# 初始化帧列表，用于存储渲染的图像
frames = []
# 初始化时间刻列表，用于记录每次迭代的时间
ticks = []
# 初始化奖励列表，用于存储每一步的奖励
rewards = []
# 初始化观测列表，用于存储每一步的观测数据
observations = []

# 获取动作规格，用于生成符合要求的随机动作
spec = env.action_spec()
# 重置环境，获取初始时间步
time_step = env.reset()

# 当物理引擎中的时间小于指定时长时，持续进行模拟
while env.physics.data.time < duration:
  # 生成随机动作，范围在动作规格的最小值和最大值之间，形状符合规格要求
  action = random_state.uniform(spec.minimum, spec.maximum, spec.shape)
  # 使用随机动作执行一步，获取新的时间步
  time_step = env.step(action)

  # 从相机 0 渲染图像，高度和宽度均为 200 像素
  camera0 = env.physics.render(camera_id=0, height=200, width=200)
  # 从相机 1 渲染图像，高度和宽度均为 200 像素
  camera1 = env.physics.render(camera_id=1, height=200, width=200)
  # 将两个相机渲染的图像水平拼接并添加到帧列表
  frames.append(np.hstack((camera0, camera1)))
  # 将当前时间步的奖励添加到奖励列表
  rewards.append(time_step.reward)
  # 将当前时间步的观测数据深拷贝后添加到观测列表
  observations.append(copy.deepcopy(time_step.observation))
  # 将当前物理引擎的时间添加到时间刻列表
  ticks.append(env.physics.data.time)

# 将帧列表转换为 HTML 视频，帧率基于环境控制时间步长
html_video = display_video(frames, framerate=1./env.control_timestep())

# 显示视频并绘制奖励和观测数据的图表
# 获取观测数据中的传感器数量
num_sensors = len(time_step.observation)

# 创建一个包含 1 + 传感器数量的子图，共享 x 轴，设置图表大小为 (4, 8)
_, ax = plt.subplots(1 + num_sensors, 1, sharex=True, figsize=(4, 8))
# 在第一个子图中绘制时间与奖励的关系图
ax[0].plot(ticks, rewards)
# 设置第一个子图的 y 轴标签为 'reward'
ax[0].set_ylabel('reward')
# 设置最后一个子图的 x 轴标签为 'time'
ax[-1].set_xlabel('time')

# 遍历观测数据中的每个键值对
for i, key in enumerate(time_step.observation):
  # 将每个观测键对应的数据转换为数组
  data = np.asarray([observations[j][key] for j in range(len(observations))])
  # 在对应子图中绘制时间与数据的图表，并添加标签
  ax[i+1].plot(ticks, data, label=key)
  # 设置对应子图的 y 轴标签为观测键名
  ax[i+1].set_ylabel(key)

# 返回生成的 HTML 视频
html_video

In [ ]:
#@title Visualizing an initial state of one task per domain in the Control Suite: 可视化 Control Suite 中每个域的一个任务的初始状态

# 创建一个字典，包含所有域及其对应的任务，从 suite.ALL_TASKS 中获取
domains_tasks = {domain: task for domain, task in suite.ALL_TASKS}
# 创建一个随机状态对象，种子设为 42，确保结果可重复
random_state = np.random.RandomState(42)
# 获取域的数量
num_domains = len(domains_tasks)
# 计算子图的列数，取域数量除以其平方根的整数部分
n_col = num_domains // int(np.sqrt(num_domains))
# 计算子图的行数，根据域数量和列数计算，考虑余数情况
n_row = num_domains // n_col + int(0 < num_domains % n_col)
# 创建一个子图网格，行数为 n_row，列数为 n_col，图表大小为 (12, 12)
_, ax = plt.subplots(n_row, n_col, figsize=(12, 12))
# 遍历所有子图，关闭坐标轴显示
for a in ax.flat:
  a.axis('off')
  # 关闭子图的网格线
  a.grid(False)

# 打印信息，显示正在遍历 Suite 中的所有域，数量为 num_domains
print(f'Iterating over all {num_domains} domains in the Suite:')
# 遍历 domains_tasks 的键值对（域和任务），使用 enumerate 获取索引
for j, [domain, task] in enumerate(domains_tasks.items()):
  # 打印当前处理的域和任务名称
  print(domain, task)

  # 加载指定域和任务的环境，传入随机状态作为任务参数
  env = suite.load(domain, task, task_kwargs={'random': random_state})
  # 重置环境，获取初始时间步
  timestep = env.reset()
  # 渲染环境的初始状态图像，高度和宽度为 200 像素，使用相机 0
  pixels = env.physics.render(height=200, width=200, camera_id=0)

  # 在对应子图中显示渲染的图像
  ax.flat[j].imshow(pixels)
  # 设置子图的标题为“域: 任务”形式
  ax.flat[j].set_title(domain + ': ' + task)

# 清除之前的输出，只保留最终的可视化结果
clear_output()

# Locomotion: 运动

## Humanoid running along corridor with obstacles: 人形机器人沿走廊跑步并避开障碍物


As an illustrative example of using the Locomotion infrastructure to build an RL environment, consider placing a humanoid in a corridor with walls, and a task specifying that the humanoid will be rewarded for running along this corridor, navigating around the wall obstacles using vision. We instantiate the environment as a composition of the Walker, Arena, and Task as follows. First, we build a position-controlled CMU humanoid walker.

- 作为一个使用 Locomotion 基础设施构建强化学习（RL）环境的说明性示例，考虑将一个人形机器人放置在一个有墙壁的走廊中，任务指定人形机器人将因沿着走廊跑步并使用视觉导航绕过墙壁障碍物而获得奖励。
- 我们将环境实例化为 Walker（行走者）、Arena（场地）和 Task（任务）的组合，具体如下。首先，我们构建一个位置控制的 CMU 人形行走者。


In [ ]:
#@title A position controlled `cmu_humanoid`: 一个位置控制的 `cmu_humanoid`

# 创建一个位置控制的 CMU 人形行走者（基于 2020 版本）
# 设置可观察选项，启用以自我为中心的摄像头
walker = cmu_humanoid.CMUHumanoidPositionControlledV2020(
    observable_options={'egocentric_camera': dict(enabled=True)})

Next, we construct a corridor-shaped arena that is obstructed by walls.
接下来，我们构建一个被墙壁阻挡的走廊形竞技场。

In [ ]:
#@title A corridor arena with wall obstacles: 一个带有墙壁障碍物的走廊场地

# 创建一个带有墙壁的走廊场地实例
arena = corridor_arenas.WallsCorridor(
    # 设置墙壁之间的间隙为 3 个单位
    wall_gap=3.,
    # 设置墙壁宽度为 2 到 3 之间的均匀分布随机值
    wall_width=distributions.Uniform(2., 3.),
    # 设置墙壁高度为 2.5 到 3.5 之间的均匀分布随机值
    wall_height=distributions.Uniform(2.5, 3.5),
    # 设置走廊宽度为 4 个单位
    corridor_width=4.,
    # 设置走廊长度为 30 个单位
    corridor_length=30.,
)

The task constructor places the walker in the arena.

In [ ]:
#@title A task to navigate the arena

task = corridor_tasks.RunThroughCorridor(
    walker=walker,
    arena=arena,
    walker_spawn_position=(0.5, 0, 0),
    target_velocity=3.0,
    physics_timestep=0.005,
    control_timestep=0.03,
)

Finally, a task that rewards the agent for running down the corridor at a specific velocity is instantiated as a `composer.Environment`.

In [ ]:
#@title The `RunThroughCorridor` environment

env = composer.Environment(
    task=task,
    time_limit=10,
    random_state=np.random.RandomState(42),
    strip_singleton_obs_buffer_dim=True,
)
env.reset()
pixels = []
for camera_id in range(3):
  pixels.append(env.physics.render(camera_id=camera_id, width=240))
PIL.Image.fromarray(np.hstack(pixels))

## Multi-Agent Soccer

Building on Composer and Locomotion libraries, the Multi-agent soccer environments, introduced in [this paper](https://arxiv.org/abs/1902.07151), follow a consistent task structure of Walkers, Arena, and Task where instead of a single walker, we inject multiple walkers that can interact with each other physically in the same scene. The code snippet below shows how to instantiate a 2-vs-2 Multi-agent Soccer environment with the simple, 5 degree-of-freedom `BoxHead` walker type.

In [ ]:
#@title 2-v-2 `Boxhead` soccer

random_state = np.random.RandomState(42)
env = soccer.load(
    team_size=2,
    time_limit=45.,
    random_state=random_state,
    disable_walker_contacts=False,
    walker_type=soccer.WalkerType.BOXHEAD,
)
env.reset()
pixels = []
# Select a random subset of 6 cameras (soccer envs have lots of cameras)
cameras = random_state.choice(env.physics.model.ncam, 6, replace=False)
for camera_id in cameras:
  pixels.append(env.physics.render(camera_id=camera_id, width=240))
image = np.vstack((np.hstack(pixels[:3]), np.hstack(pixels[3:])))
PIL.Image.fromarray(image)

 It can trivially be replaced by e.g. the `WalkerType.ANT` walker:

In [ ]:
#@title 3-v-3 `Ant` soccer

random_state = np.random.RandomState(42)
env = soccer.load(
    team_size=3,
    time_limit=45.,
    random_state=random_state,
    disable_walker_contacts=False,
    walker_type=soccer.WalkerType.ANT,
)
env.reset()

pixels = []
cameras = random_state.choice(env.physics.model.ncam, 6, replace=False)
for camera_id in cameras:
  pixels.append(env.physics.render(camera_id=camera_id, width=240))
image = np.vstack((np.hstack(pixels[:3]), np.hstack(pixels[3:])))
PIL.Image.fromarray(image)

# Manipulation: 操作

The `manipulation` module provides a robotic arm, a set of simple objects, and tools for building reward functions for manipulation tasks.

In [ ]:
#@title Listing all `manipulation` tasks{vertical-output: true}

# `ALL` is a tuple containing the names of all of the environments in the suite.
print('\n'.join(manipulation.ALL))

In [ ]:
#@title Listing `manipulation` tasks that use vision{vertical-output: true}
print('\n'.join(manipulation.get_environments_by_tag('vision')))

In [ ]:
#@title Loading and simulating a `manipulation` task{vertical-output: true}

env = manipulation.load('stack_2_of_3_bricks_random_order_vision', seed=42)
action_spec = env.action_spec()

def sample_random_action():
  return env.random_state.uniform(
      low=action_spec.minimum,
      high=action_spec.maximum,
  ).astype(action_spec.dtype, copy=False)

# Step the environment through a full episode using random actions and record
# the camera observations.
frames = []
timestep = env.reset()
frames.append(timestep.observation['front_close'])
while not timestep.last():
  timestep = env.step(sample_random_action())
  frames.append(timestep.observation['front_close'])
all_frames = np.concatenate(frames, axis=0)
display_video(all_frames, 30)